# How to debug your LLM apps

Like building any type of software, at some point you'll need to debug when building with LLMs. A model call will fail, or model output will be misformatted, or there will be some nested model calls and it won't be clear where along the way an incorrect output was created.

There are three main methods for debugging:

- Verbose Mode: This adds print statements for "important" events in your chain.
- Debug Mode: This add logging statements for ALL events in your chain.
- LangSmith Tracing: This logs events to [LangSmith](https://docs.smith.langchain.com/) to allow for visualization there.

|                        | Verbose Mode | Debug Mode | LangSmith Tracing |
|------------------------|--------------|------------|-------------------|
| Free                   | ✅            | ✅          | ✅                 |
| UI                     | ❌            | ❌          | ✅                 |
| Persisted              | ❌            | ❌          | ✅                 |
| See all events         | ❌            | ✅          | ✅                 |
| See "important" events | ✅            | ❌          | ✅                 |
| Runs Locally           | ✅            | ✅          | ❌                 |


## Tracing

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```

Let's suppose we have an agent, and want to visualize the actions it takes and tool outputs it receives. Without any debugging, here's what we see:

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>


In [2]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

In [3]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_tavily import TavilySearch

tools = [TavilySearch(max_results=5, topic="general")]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': "The 2023 film **Oppenheimer** was directed by **Christopher Nolan**.\n\nChristopher Nolan was born on **July 30, 1970**. To calculate his age in days as of today:\n\n1. First, determine the total number of days from his birthdate to today.\n2. Use the formula: \\[ \\text{Age in days} = (\\text{Current Year} - \\text{Birth Year}) \\times 365 + \\text{Extra Days for Leap Years} + \\text{Days from Birthday to Today's Date} \\]\n\nLet's calculate:\n\n- From July 30, 1970, to July 30, 2023, is 53 years.\n- From July 30, 2023, to December 7, 2023, is 130 days.\n\nLeap years between 1970 and 2023 (every 4 years, except century years not divisible by 400):\n1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020. That's 13 leap years.\n\nSo, his age in days is:\n\\[ 53 \\times 365 + 13 + 130 = 19345 + 13 + 130 = 19488 \\text{ days} \\]\n\nChristopher Nolan is **19,488 days old** 

We don't get much output, but since we set up LangSmith we can easily see what happened under the hood:

https://smith.langchain.com/public/a89ff88f-9ddc-4757-a395-3a1b365655bf/r

## `set_debug` and `set_verbose`

If you're prototyping in Jupyter Notebooks or running Python scripts, it can be helpful to print out the intermediate steps of a chain run.

There are a number of ways to enable printing at varying degrees of verbosity.

Note: These still work even with LangSmith enabled, so you can have both turned on and running at the same time


### `set_verbose(True)`

Setting the `verbose` flag will print out inputs and outputs in a slightly more readable format and will skip logging certain raw outputs (like the token usage stats for an LLM call) so that you can focus on application logic.

In [4]:
from langchain.globals import set_verbose

set_verbose(True)
agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)



> Entering new AgentExecutor chain...

Invoking: `tavily_search` with `{'query': 'director of the 2023 film Oppenheimer'}`


{'query': 'director of the 2023 film Oppenheimer', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Oppenheimer (film) - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Oppenheimer_(film)', 'content': "Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Toggle the table of contents Contents move to sidebar hide (Top) 1 Plot 2 Cast 3 ProductionToggle Production subsection 3.1 Development 3.2 Writing 3.3 Casting 3.4 Filming 3.5 Post-production 4 Music 5 Marketing 6 ReleaseToggle Release subsection 6.1 Theatrical 6.1.1 Classifications and censorship 6.1.2 Bhagavad Gita controversy 6.2 Home media 7 ReceptionToggle Reception subsection 7.1 Box office 7.1.1 United States and Canada 7.1.2 Japan 7.1.3 Other territories 7.2 Critical response 7.3 Influence o

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': 'Christopher Nolan was born on July 30, 1970. As of today, June 15, 2025, he is 19,944 days old.'}

### `set_debug(True)`

Setting the global `debug` flag will cause all LangChain components with callback support (chains, models, agents, tools, retrievers) to print the inputs they receive and outputs they generate. This is the most verbose setting and will fully log raw inputs and outputs.

In [5]:
from langchain.globals import set_debug

set_debug(True)
set_verbose(False)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_executor.invoke(
    {"input": "Who directed the 2023 film Oppenheimer and what is their age in days?"}
)

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Who directed the 2023 film Oppenheimer and what is their age in days?"
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad>] Entering Chain run with input:
{
  "input": ""
}
[chain/start] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": ""
}
[chain/end] [chain:AgentExecutor > chain:RunnableSequence > chain:RunnableAssign<agent_scratchpad> > chain:RunnableParallel<agent_scratchpad> > chain:RunnableLambda] s] Exiti

{'input': 'Who directed the 2023 film Oppenheimer and what is their age in days?',
 'output': "The 2023 film **Oppenheimer** was directed by **Christopher Nolan**.\n\nChristopher Nolan was born on **July 30, 1970**. To calculate his age in days as of today:\n\n1. First, determine the total number of days from his birthdate to today.\n2. Use the formula: \\[ \\text{Age in days} = \\text{Current Date} - \\text{Birth Date} \\]\n\nLet's calculate:\n\n- Birthdate: July 30, 1970\n- Today's Date: December 7, 2023\n\nUsing a date calculator or similar method, we find that Christopher Nolan is approximately **19,480 days old** as of today."}